In [1]:
import sys
sys.path.append('src/')
from caiman_interface import *

# <em>Ca</em><font color="grey">lcium</font> <em>Im</em><font color="grey">aging</font> <em>An</em><font color="grey">alysis</font>
### Jupyter Notebook Interface (alpha version)

##### Manage Context
The context is the data that is saved during each step of the analysis. You can save all of your work as a `context` file with the extension `.pkl`. For example, if you finish the CNMF-E analysis step, you can save your context, and then re-load it later to view the results interactively here. Please note the context files are large, so only save if necessary.

In [2]:
%matplotlib qt
display(app_ui)

In [3]:
fr = 30                             # imaging rate in frames per second
decay_time = 0.4                    # length of a typical transient in seconds

# motion correction parameters
splits_rig = 56             # for parallelization split the movies in  num_splits chuncks across time
strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thresh = 0.8          # merging threshold, max correlation allowed
rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6             # amount of overlap between the patches in pixels
K = 4                       # number of components per patch
gSig = [4, 4]               # expected half size of neurons
method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
is_dendrites = False        # flag for analyzing dendritic data

# parameters for component evaluation
min_SNR = 2.0               # signal to noise ratio for accepting a component
rval_thr = 0.7              # space correlation threshold for accepting a component
cnn_thr = 0.7               # threshold for CNN based classifier
opts_dict = {
            'fr': fr,
            'decay_time': decay_time,
            'splits_rig': splits_rig,
            'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid,
            'p': 1,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'rolling_sum': True,
            'is_dendrites': is_dendrites,
            'min_SNR': min_SNR,
            'rval_thr': rval_thr,
            'use_cnn': False,
            'min_cnn_thr': cnn_thr}
opts = params.CNMFParams(params_dict=opts_dict)
opts.set('temporal', {'p': 0})
context.cnm.estimates.filter_components(reshape_Yr(*context.YrDT), params=opts)

AttributeError: 'NoneType' object has no attribute 'estimates'

### Notes

##### Motion Correction Parameters:
- Grouped?  If your folder contains movies from the _same_ experiment, i.e. if your recording software produces multiple segments of a single recording, then select "Group" so these will be combined in the analysis to a single movie. If your folder contains movies from different recordings, select "Independent" so they will be run separately.
- Downsample factors are percentages from 0 to 1. If you enter 0.5 for height and width, then the movie resolution will be halved. E.g. if you have a 500px by 500px movie, it will be downsampled to 250px by 250px. You can also downsample in the time domain, i.e. if your movie has 1000 frames, entering 0.5 will downsample the frames to 500 frames. This should only be done if you're recording at a high frame rate.
- "High Pass" setting should be left at default value.
- Motion Correction has two modes: rigid and non-rigid. Rigid is much faster. Non-rigid should be used when the movie has non-rigid (elastic) motions/deformations. This can sometimes happen given with _in vivo_ brain recordings since the brain is an elastic organ.
- MC Results: This section is under development. Currently you can only view a side-by-side of the original movie with the motion corrected movie. The movies will pop-up in a separate window. Press the __Q__ key on your keyboard to close the movie window.

##### CNMF-E Parameters:
- Patches? CNMF-E can run in two modes: patches or single field of view (FOV). Patches mode will break up each frame into multiple pieces and process them independently. On computers with multiple processors, this can be much faster. However, patches mode can produce unwanted reconstruction artifacts when the pieces are poorly re-assembled.
- You can downsample in the CNMF-E stage, spatially (height and width) and temporally (frames/time). However, if you already downsampled during motion correction, you probably should not downsample again.
- __K__: K is the estimated number of neurons in your field of view. This parameter helps the algorithm detect the correct number of neurons. It does not need to be highly accurate; an over-estimate is better.
- __gSig__: This is the estimated half-width (radius) of each neuron in the movie, in pixel values. The default value will work in most cases.
- __gSiz__: This is related to gSig and typically the default value will work fine.

##### CNMF-E Results Viewer
- This interface allows you to look at a "Z-projection" (i.e. flattened video) of the data to see how well the CNMF-E algorithm did at detecting neurons and avoiding noise. In the top left is a the Z-projected image with red dots denoting the neurons it detected. 
- In general, CNMF-E _will_ think some noise is a neuron. In that case, you need to look through the traces and at the image panels to see which cells are noise and press the `Delete ROI` button. This will not modify the `context`. This just excludes those ROIs if you download the data as a file.
- Check the `Use dF/F` checkbox if you want to convert the signals to delta fluorescence / fluorescence. This is a necessary step if you want to be able to compare the signals between animals and recording sessions.

In [ ]:
#clean_up(stop_server=True)